In [1]:
import datetime
import numpy
import netCDF4
from osgeo import gdal, osr
import ogr
import os
import pandas as pd
from joblib import Parallel, delayed
import multiprocessing as mp
import collections
import logging
import time

logging.basicConfig(level=logging.DEBUG, filename=r'P:\projects\SILO\siloLog.txt')

#example
#start = '2015-01-01'
#end = '2015-03-23'
#sd = dt.strptime(start,'%Y-%m-%d')
#ed = dt.strptime(end,'%Y-%m-%d')
#delta = ed - sd
#for i in range(delta.days+1):
#    dict[sd + td(days=i)] = <something>

reqStart = '1980-01-01'
#reqEnd = '1981-12-31'
reqEnd = '1980-01-14'

inputGeoCoordsFile = r'P:\projects\SILO\Wes\MNDB_geocoords.csv'
outputDir = r'P:\projects\SILO\Wes\testOut2'
siloNetCDFDir = r'P:\projects\SILO\Wes\mwet'

varNameInSilo = 'et_morton_wet'
varForColName = 'pet'

#Col names for GeoCoords CSV
catGrpName = 'CATCHGROUP'
latName = 'SILO_LAT'
longName = 'SILO_LONG'
countName = 'COUNT'

In [2]:
#Base output dir
if not os.path.exists(outputDir):
    os.makedirs(outputDir)
    print("Made dir: " + outputDir)


Made dir: P:\projects\SILO\Wes\testOut2


In [3]:
#Read in GeoCoords
GeoCoordsVals = {}

geoCoordsDF = pd.read_csv(inputGeoCoordsFile)
#for index, row in geoCoordsDF.iterrows():
#    if not row[catGrpName] in GeoCoordsVals:
#        #print('Adding: ' + str(row[catFileRasID]) + ' and ' + str(row[catFileCatName]))
#        GeoCoordsVals[row[catGrpName]] = row[catFileCatName]

allCats = geoCoordsDF[catGrpName].unique().tolist()
colNames = [varForColName + ' for ' + cat for cat in allCats]
print(len(allCats))

213


In [4]:

sd = datetime.datetime.strptime(reqStart,'%Y-%m-%d')
ed = datetime.datetime.strptime(reqEnd,'%Y-%m-%d')
##delta = ed - sd

sYear = sd.year
eYear = ed.year

allYears = []
for y in range(sYear, eYear + 1):
    if not y in allYears:
        allYears.append(y)

#date_today = datetime.now()
#days = pd.date_range(date_today, date_today + timedelta(7), freq='D')
date_rng = pd.date_range(sd, ed, freq='D')
#print(str(type(days[0])))
daysAsList = date_rng.tolist()
#mainDF = pd.DataFrame({'Date': days})
mainDF = pd.DataFrame(date_rng, columns=['tempDate'])
#mainDF = mainDF.set_index('Date')

#Getting date as index????
mainDF['Date'] = pd.to_datetime(mainDF['tempDate'])
mainDF = mainDF.set_index('Date')
mainDF.drop(['tempDate'], axis=1, inplace=True)


for col in colNames:
    mainDF[col] = ''

resultsDict = {}
#for d in daysAsList:
#    resultsDict[d] = {}



mainDF.head()
#print(mainDF)

,pet for SC #0,pet for SC #1,pet for SC #2,pet for SC #3,pet for SC #4,pet for SC #5,pet for SC #6,pet for SC #7,pet for SC #8,pet for SC #9,...,pet for SC #113,pet for SC #114,pet for SC #115,pet for SC #116,pet for SC #117,pet for SC #118,pet for SC #119,pet for SC #120,pet for SC #121,pet for SC #122
Date,,,,,,,,,,,,,,,,,,,,,
1980-01-01,,,,,,,,,,,...,,,,,,,,,,
1980-01-02,,,,,,,,,,,...,,,,,,,,,,
1980-01-03,,,,,,,,,,,...,,,,,,,,,,
1980-01-04,,,,,,,,,,,...,,,,,,,,,,
1980-01-05,,,,,,,,,,,...,,,,,,,,,,


In [5]:
#class to contain a raster band
DailyRasterBand = collections.namedtuple('DailyRasterBand', [
    'dateOfRaster',
    'RasterArray',
    'geoTransObject',
    'scaleFactor',
    'noDataVal'
])

In [8]:
def processBand(number, allBands):

    result = {}
    srcband = allBands[number]

    #logging.info('Checking this item: ' + str(number) + ' is a ' + str(type(srcband.RasterBand))
    #             + 'of date ' + srcband.dateOfRaster)
   
    #scaleFactor = 1.0
        
    #sf = srcband.RasterBand.GetMetadataItem(varNameInSilo + '_scale_factor')
    #nd = srcband.RasterBand.GetMetadataItem(varNameInSilo + '__FillValue')
    #nd2 = srcband.RasterBand.GetNoDataValue()

        
    scaleFactor = float(srcband.scaleFactor)
    rawNoData = float(srcband.noDataVal)
    
    fakeSiloGT = srcband.geoTransObject
    #siloArray = srcband.RasterBand.ReadAsArray()
    ##just make sure there's no gaps? Not necessary?
    ##siloMask = numpy.ma.masked_where(siloArray == nd, siloArray)
    
    print(srcband.dateOfRaster + ' ' + str(len(srcband.RasterArray)))
    #logging.info('Checking this array: ' + str(siloArray))
        
    #print(dateForFileName + ': ' + str(siloMask[500,500] * scaleFactor))
    
    #Invert Y-axis values...
    #Really important
    invertArray = srcband.RasterArray[::-1,:]
        
    for thisCat in allCats:
            
        colName = varForColName + ' for ' + thisCat
        catTodaySum = 0.0
        catTodayCount = 0.0
            
        catDF = geoCoordsDF.loc[geoCoordsDF[catGrpName] == thisCat]
            
        for index, row in catDF.iterrows():
                
            longVal = float(row[longName])
            latVal = float(row[latName])
            countVal = int(row[countName])
                
            geoXcell = int((longVal - fakeSiloGT[0]) / fakeSiloGT[1]) #x pixel
            geoYcell = int((latVal - fakeSiloGT[3]) / fakeSiloGT[5]) #y pixel
                
                
            catTodaySum += (invertArray[geoYcell,geoXcell] * scaleFactor) * countVal
            catTodayCount += countVal
        
        
        #Calc average
        avVal = 0
        if catTodayCount > 0:
            avVal = catTodaySum / catTodayCount
            
        result.update({thisCat:avVal})
    
    #Try and jam into dataframe
    #for cat,val in result.items():
    #    colName = varForColName + ' for ' + cat
    #    mainDF[srcband.dateOfRaster][colName] = val
    
    resultsDict[srcband.dateOfRaster] = result
    
    return
    #return {
    #    'name': item.name,
    #    'age': 2017 - item.born
    #}

In [9]:
#load the data
#print(allYears)
print('started at: ' + str(datetime.datetime.now()))
data = (['a', '2'], ['b', '4'], ['c', '6'], ['d', '8'],['e', '1'], ['f', '3'], ['g', '5'], ['h', '7'])
numbers = [5, 10, 15, 20, 25]
procs = []

for thisYear in allYears:
    
    siloFile = os.path.join(siloNetCDFDir, str(thisYear) + '.' + varNameInSilo + '.nc')
    print('Processing: ' + siloFile)
    #print(type(thisYear))
    
    #Need the gdal version of silo to give us a few params, but will close of netCDF version pronto...
    siloDS = gdal.Open(siloFile)
    
    ##Need this to calc pixel size as well as work out order of y-axis
    netCDFData = netCDF4.Dataset(siloFile, 'r')

    #Grab the info we need to build GeoTransform object
    #Checking for order of y-axis coordinates, would be needed if writing raster form Numpy array
    
    lats = netCDFData.variables['lat']
    lons = netCDFData.variables['lon']
    times = netCDFData.variables['time']
    
    first_lat = lats[:1]
    last_lat = lats[len(lats)-1:]
    #If first_lat is LESS THAN last_lat, numpy array would be written to raster upside down...
    
    first_lon = lons[:1]
    last_lon = lons[len(lons)-1:]
    
    colCount = siloDS.RasterXSize
    rowCount = siloDS.RasterYSize
    
    pixWidth = (last_lon - first_lon) / (siloDS.RasterXSize - 1)
    pixHeight = (last_lat - first_lat) / (siloDS.RasterYSize - 1)

    if first_lat < last_lat:
        #adjust for our use
        pixHeight *= -1
    
    
    siloLeft = min(lons) - (0.5 * pixWidth)#adjusting to get cell edge, not centre
    siloTop = max(lats) - (0.5 * pixHeight)#adjusting to get cell edge, not centre
    
    netCDFData.close()
    
    fakeSiloGT = [siloLeft, pixWidth, 0, siloTop, 0, pixHeight]
    
    #Jan 1
    siloDStart = datetime.datetime(year=thisYear, month=1, day=1)
    
    #theDataSet.RasterCount is the number of days we have
    #remembering that 'range' give a STOP value that isn't included
    #however, depsite 'band' starting at zero, band IDs will start at 1!
    
    allbands = []
    for band in range(0, siloDS.RasterCount):
        
        dateNow = siloDStart + datetime.timedelta(days=band)
        dateForFileName = str(dateNow.year) + str(dateNow.month).zfill(2) + str(dateNow.day).zfill(2)
        dateTimeStampKey = str(dateNow.year) + '-' + str(dateNow.month).zfill(2) + '-' + str(dateNow.day).zfill(2)
        
        if not dateNow in daysAsList:
            continue
        
        #print("Getting: " + dateTimeStampKey + " from Idx: " + str(band))
        theBand = siloDS.GetRasterBand(band + 1)
        theSF = theBand.GetMetadataItem(varNameInSilo + '_scale_factor')
        theND = theBand.GetMetadataItem(varNameInSilo + '__FillValue')
        
        scaleFactor = float(theSF)
        rawNoData = float(theND)

        
        allbands.append(DailyRasterBand(dateOfRaster=dateTimeStampKey,
                                        RasterArray=theBand.ReadAsArray(), geoTransObject=fakeSiloGT,
                                       scaleFactor=scaleFactor, noDataVal=rawNoData))
    
    num_cores = mp.cpu_count()
    procList = list(range(1, siloDS.RasterCount + 1))
    #Parallel(n_jobs=num_cores)(delayed(doubler) for i in numbers)#in enumerate(numbers)#, siloDS=siloDS
    #Parallel(n_jobs=num_cores, require='sharedmem')(delayed(doubler)(i) for i in range(5))
    Parallel(n_jobs=num_cores, require='sharedmem')(delayed(processBand)(i, allbands) for i in range(0, len(allbands)))
    #print(ff)
    #print(shared_set)
    
    #pool = mp.Pool(mp.cpu_count())
    #print(str(mp.cpu_count()))
    ##results = [pool.apply(grabDailyVals2, args=(row)) for row in procList]
    #pool.close() 
    
#mainDF.head()
print('finished at: ' + str(datetime.datetime.now()))

started at: 2019-07-23 09:31:16.196120
Processing: P:\projects\SILO\Wes\mwet\1980.et_morton_wet.nc
1980-01-01 681
1980-01-02 6811980-01-03 681
1980-01-04 681
1980-01-05 681
1980-01-06 681

1980-01-07 681
1980-01-08 681
1980-01-09 681
1980-01-10 681
1980-01-11 681
1980-01-12 681
1980-01-13 681
1980-01-14 681
finished at: 2019-07-23 09:31:50.413361


In [10]:
for t,data in resultsDict.items():
    for col,val in data.items():
        colName = varForColName + ' for ' + col
        mainDF.loc[t][colName] = val

In [11]:
mainDF.head()

,pet for SC #0,pet for SC #1,pet for SC #2,pet for SC #3,pet for SC #4,pet for SC #5,pet for SC #6,pet for SC #7,pet for SC #8,pet for SC #9,...,pet for SC #113,pet for SC #114,pet for SC #115,pet for SC #116,pet for SC #117,pet for SC #118,pet for SC #119,pet for SC #120,pet for SC #121,pet for SC #122
Date,,,,,,,,,,,,,,,,,,,,,
1980-01-01,6.1172,6.07922,6.22556,6.0912,6.25795,6.30164,6.24845,5.93978,6.28013,6.35329,...,3.91357,4.03692,4.47234,3.70863,4.97597,4.02607,3.66122,4.14221,3.32605,3.13004
1980-01-02,7.39102,6.94727,7.3036,7.39851,7.19553,7.27585,7.07919,7.36526,6.98245,7.13228,...,3.22187,2.94344,3.54043,2.91582,3.97597,3.06976,2.52954,3.19759,2.47047,2.66155
1980-01-03,7.67772,7.2489,7.68216,7.66543,7.59772,7.64225,7.45435,7.58615,7.25795,7.50472,...,5.61001,4.49747,5.69421,5.05631,5.87597,4.99279,3.7758,5.36738,3.77314,4.62329
1980-01-04,4.59176,5.09405,4.68609,4.4827,4.79086,4.71014,4.90185,4.3715,5.04793,4.81685,...,3.61319,4.01115,3.87643,3.6775,4,3.85396,4.09766,3.83774,3.7431,3.36638
1980-01-05,2.45046,2.92911,2.51859,2.4,2.65209,2.59409,2.78162,2.38211,2.96386,2.72437,...,2.32873,2.27079,2.49218,2.24243,2.67597,2.31085,2.09507,2.36722,2.10473,2.11984


In [12]:
mainDF.to_csv(r'P:\projects\SILO\testMulti2.csv', index=True)